<a href="https://colab.research.google.com/github/sudarshan0894/Project_2024/blob/main/TextClassicatifation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.15.0


In [14]:
!kaggle datasets download -d dushyantv/consumer_complaints

Dataset URL: https://www.kaggle.com/datasets/dushyantv/consumer_complaints
License(s): CC0-1.0
 94% 134M/142M [00:01<00:00, 117MB/s]
100% 142M/142M [00:01<00:00, 121MB/s]


In [16]:
import zipfile
import os

# Extract the dataset
with zipfile.ZipFile('consumer_complaints.zip', 'r') as zip_ref:
    zip_ref.extractall('consumer_complaints')

# Load the dataset
df = pd.read_csv('consumer_complaints/Consumer_Complaints.csv', encoding='utf-8')

# Display the first few rows
print(df.head())


  Date received           Product     Sub-product  \
0    03-12-2014          Mortgage  Other mortgage   
1    10-01-2016  Credit reporting             NaN   
2    10/17/2016     Consumer Loan    Vehicle loan   
3    06-08-2014       Credit card             NaN   
4    09/13/2014   Debt collection     Credit card   

                                      Issue                   Sub-issue  \
0  Loan modification,collection,foreclosure                         NaN   
1    Incorrect information on credit report              Account status   
2                Managing the loan or lease                         NaN   
3                                Bankruptcy                         NaN   
4                     Communication tactics  Frequent or repeated calls   

                                  Consumer Complaint  \
0                                                NaN   
1  I have outdated information on my credit repor...   
2  I purchased a new car on XXXX XXXX. The car de...   
3     

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025010 entries, 0 to 1025009
Data columns (total 19 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Date received                 1025010 non-null  object 
 1   Product                       1025010 non-null  object 
 2   Sub-product                   789840 non-null   object 
 3   Issue                         1025010 non-null  object 
 4   Sub-issue                     528853 non-null   object 
 5   Consumer Complaint            277814 non-null   object 
 6   Company Public Response       318364 non-null   object 
 7   Company                       1025010 non-null  object 
 8   State                         1012650 non-null  object 
 9   ZIP code                      1008292 non-null  object 
 10  Tags                          141588 non-null   object 
 11  Consumer consent provided?    491911 non-null   object 
 12  Submitted via               

In [17]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer Complaint,Company Public Response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date Sent to Company,Company Response to Consumer,Timely response?,Consumer disputed?,Complaint ID,Unnamed: 18
0,03-12-2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217,NaN
1,10-01-2016,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,10-05-2016,Closed with explanation,Yes,No,2141773,NaN
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No,2163100,NaN
3,06-08-2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06-10-2014,Closed with explanation,Yes,Yes,885638,NaN
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760,NaN


In [26]:
col= ["Consumer Complaint","Product"]
df=df[col]
df = df[pd.notnull(df['Consumer Complaint'])]
df.columns = ['Consumer Complaint','Product']
df.head()

,Consumer Complaint,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection


In [28]:
df.isnull().sum()

Consumer Complaint    0
Product               0
dtype: int64

In [29]:
df["Product"].value_counts()

Product
Debt collection                                                                 63268
Credit reporting, credit repair services, or other personal consumer reports    49006
Mortgage                                                                        43837
Credit reporting                                                                31593
Credit card                                                                     18842
Student loan                                                                    16689
Bank account or service                                                         14887
Credit card or prepaid card                                                     10659
Consumer Loan                                                                    9474
Checking or savings account                                                      6489
Money transfer, virtual currency, or money service                               3089
Vehicle loan or lease                         

In [30]:
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

Train size: 222251
Test size: 55563


In [31]:
train_posts = df['Consumer Complaint'][:train_size]
train_tags = df['Product'][:train_size]

test_posts = df['Consumer Complaint'][train_size:]
test_tags = df['Product'][train_size:]

In [33]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [34]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [35]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [36]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (222251, 1000)
x_test shape: (55563, 1000)
y_train shape: (222251, 18)
y_test shape: (55563, 18)


In [37]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
batch_size = 32
epochs = 5

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
6251/6251 [==============================] - 71s 11ms/step - loss: 0.8660 - accuracy: 0.7270 - val_loss: 1.2360 - val_accuracy: 0.4879
Epoch 2/5
6251/6251 [==============================] - 68s 11ms/step - loss: 0.7471 - accuracy: 0.7582 - val_loss: 1.3029 - val_accuracy: 0.5008
Epoch 3/5
6251/6251 [==============================] - 69s 11ms/step - loss: 0.6890 - accuracy: 0.7756 - val_loss: 1.3537 - val_accuracy: 0.4933
Epoch 4/5
6251/6251 [==============================] - 71s 11ms/step - loss: 0.6424 - accuracy: 0.7910 - val_loss: 1.3289 - val_accuracy: 0.5221
Epoch 5/5
6251/6251 [==============================] - 65s 10ms/step - loss: 0.6011 - accuracy: 0.8038 - val_loss: 1.4384 - val_accuracy: 0.5019


In [39]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print(score)
print('Test score:', score[0])
print('Test accureacy:', score[1])

1737/1737 [==============================] - 8s 5ms/step - loss: 1.5376 - accuracy: 0.4827
[1.5376152992248535, 0.48265931010246277]
Test score: 1.5376152992248535
Test accureacy: 0.48265931010246277


In [41]:
text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts.iloc[i][:50], "...")
    print('Actual label:'+ test_tags.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

1/1 [==============================] - 0s 91ms/step
On XXXX/XXXX/XXXX, I was contacted by Cavalry rega ...
Actual label:Debt collection
Predicted label: Credit card

1/1 [==============================] - 0s 22ms/step
XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX MN, X ...
Actual label:Debt collection
Predicted label: Debt collection

1/1 [==============================] - 0s 23ms/step
I 'm currently having hardship and I ca n't make r ...
Actual label:Payday loan, title loan, or personal loan
Predicted label: Debt collection

1/1 [==============================] - 0s 21ms/step
I tried to set up a credit freeze with Equifax on  ...
Actual label:Credit reporting, credit repair services, or other personal consumer reports
Predicted label: Credit reporting

1/1 [==============================] - 0s 20ms/step
XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, CA XXXX S ...
Actual label:Debt collection
Predicted label: Debt collection

1/1 [==============================] - 0s 23ms/step
I applied for 